# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations
from scipy.ndimage import affine_transform
import SimpleITK as sitk
import matplotlib.pyplot as plt


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation, affine_transform_volume
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


In [4]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

In [13]:
moving_name = 'AtlasV8'
fixed_name = 'Allen'
moving_all = list_coms(moving_name, scaling_factor=10)
fixed_all = list_coms(fixed_name, scaling_factor=10)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)

moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(common_keys))

37


In [9]:
#transformation_matrix = get_affine_transformation(moving_name=moving_name, fixed_name=fixed_name, 
#                                                  scaling_factor=1)
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(transformation_matrix)

[[ 9.77294500e-01 -4.23063531e-02  1.32610539e-02 -1.39281947e+03]
 [ 1.31771269e-01  1.21350489e+00  7.29518726e-02 -4.33481578e+03]
 [ 4.98328669e-03 -1.48360506e-02  1.14938824e+00 -2.94630354e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [15]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = apply_affine_transform(moving0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 disk to Allen RMS 237.0680595008574
# MD585 to MD589 RMS 18.2658167690059

RMS 4106.390618253809


In [11]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#com = [1095, 392, 519]
print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
transformed_structure = apply_affine_transform(com, transformation_matrix)
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
diff = transformed_structure - fixed_all[structure]
print(f'{moving_name}->{fixed_name} {structure} {np.round(diff)}')


AtlasV8 SC non trans [10949.  3924.  5199.]
AtlasV8 SC apply trans [9211. 2249. 5677.]
Allen SC [9140. 2388. 5692.]
AtlasV8->Allen SC [  71. -139.  -15.]


In [16]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)

,structure,AtlasV8,Allen,transformed,difference,sumsquares
Index,,,,,,
12,RtTg,"[1129.0, 691.0, 510.0]","[950.0, 532.0, 569.0]","[-312.0, -3310.0, 287.0]","[-1262.0, -3842.0, -282.0]",4053.9820
29,AP,"[1439.0, 622.0, 512.0]","[1265.0, 501.0, 569.0]","[-6.0, -3353.0, 292.0]","[-1271.0, -3854.0, -278.0]",4068.0434
1,VLL_R,"[1122.0, 649.0, 674.0]","[947.0, 517.0, 750.0]","[-314.0, -3350.0, 476.0]","[-1261.0, -3868.0, -274.0]",4077.3102
10,Sp5C_L,"[1496.0, 676.0, 382.0]","[1283.0, 590.0, 398.0]","[46.0, -3289.0, 141.0]","[-1237.0, -3879.0, -257.0]",4079.5045
34,LC_R,"[1231.0, 561.0, 604.0]","[1071.0, 428.0, 665.0]","[-205.0, -3448.0, 397.0]","[-1276.0, -3876.0, -268.0]",4089.4303
13,4N_R,"[1134.0, 528.0, 547.0]","[959.0, 377.0, 595.0]","[-300.0, -3505.0, 332.0]","[-1259.0, -3882.0, -263.0]",4089.6161
5,LC_L,"[1234.0, 568.0, 437.0]","[1071.0, 428.0, 473.0]","[-205.0, -3451.0, 205.0]","[-1276.0, -3879.0, -268.0]",4091.9669
19,3N_R,"[1099.0, 528.0, 537.0]","[910.0, 379.0, 586.0]","[-334.0, -3510.0, 321.0]","[-1245.0, -3890.0, -266.0]",4092.6363
0,4N_L,"[1137.0, 528.0, 498.0]","[959.0, 378.0, 543.0]","[-298.0, -3508.0, 275.0]","[-1257.0, -3886.0, -268.0]",4092.8722


In [ ]:
outpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK55/com'
for structure, com in moving_all.items():
    comfile = structure + '.txt'
    compath = os.path.join(outpath, comfile)
    np.savetxt(compath, com)

In [ ]:
um = 50
registration_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
base_com_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data'
for brain in [moving_name, fixed_name]:
    brain_point_path = os.path.join(registration_path, brain, f'{brain}_{um}um_sagittal.pts')
    brain_com_path = os.path.join(base_com_path, brain, 'com')
    comfiles = sorted(os.listdir(brain_com_path))
    with open(brain_point_path, 'w') as f:
        f.write('point\n')
        f.write(f'{len(common_keys)}\n')
        for comfile in comfiles:
            structure = comfile.replace('.txt','')
            if structure in common_keys:
                #print(structure)
                compath = os.path.join(brain_com_path, comfile)
                x,y,z = np.loadtxt(compath)
                f.write(f'{round(x/um,4)} {round(y/um,4)} {round(z/um,4)}')
                f.write('\n')


In [ ]:
sc_com = np.array([1.119232944161371051e+04,3.845036998545344431e+03,4.365074759237448234e+03])
scale = np.array([0.452, 0.452, 20])
from_disk = sc_com/scale
print(from_disk)
from_program = np.array([24818.103954839404, 8748.195805110985, 222.0769230769231])
from_disk - from_program

In [ ]:
import math
print(0.00274 - 0.00272)
zresolution = 20
M_UM_SCALE = 1000000
test_redundant_z = []
for z_index, section in enumerate(range(136,177)):
    z = section * zresolution / M_UM_SCALE
    current_z = section
    test_redundant_z.append(current_z)
    try:
        pre_z = test_redundant_z[z_index-1]
    except IndexError:
        pre_z = current_z
        
    if current_z != pre_z:
        print('OK', section, z)
    else:
        print('BAD', section, z)